In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import LdaModel
from gensim import corpora, models
from gensim.corpora import Dictionary
from copy import copy
from nltk.stem import PorterStemmer
st=PorterStemmer()
import pymongo

/home/li/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
client = pymongo.MongoClient(host="192.168.0.106", port=29197)
client["github"].authenticate("github", "git332", "github")
db = client["github"]

In [4]:
db1 = db.all_step4
db_date = list(db1.find())

In [5]:
npm_tag_map_tpl = {"npm_"+x["name"]:0 for x in db.npm_tag_stem_top100.find()}
pkg_tag_map_tpl = {"pkg_"+x["name"]:0 for x in db.composer_tag_stem_top100.find()}
pypi_tag_map_tpl = {"pypi_"+x["name"]:0 for x in db.pypi_tag_stem_top100.find()}
tag_map_tpl = {x["name"]:0 for x in db.project_tag_more_than_100.find()} # 需要提取词干

In [6]:
lang_list = [x["name"] for x in db.project_lang_30.find()]

In [7]:
lang_map = {x:0 for x in lang_list}

In [8]:
d_list = []
train_data = pd.DataFrame()
rec_list = []
for d in tqdm(db_date):
# for d in tqdm(db1.find({"topic":"python"})):
    lang_map = {x:0 for x in lang_list}
    npm_tag_map = copy(npm_tag_map_tpl)
    pkg_tag_map = copy(pkg_tag_map_tpl)
    pypi_tag_map = copy(pypi_tag_map_tpl)
    tag_map = copy(tag_map_tpl)
    doc = {}
    doc["project_link"] = d["link"]
    
    need_add = False

    # tag
    has_tag = False
    for t in d["topics"]:
        tag = st.stem(t)
        if tag in tag_map:
            tag_map[tag]+=1
            has_tag = True
            
    if not has_tag:
        continue
        

        
    # readme
    if "readme_parser" in d:
        doc["file_readme"] = d["readme_parser"]
        need_add = True
    else:
        doc["file_readme"] = ""
        
        
    # 包管理工具
    if "package_paser" in d:
        doc["file_npm"] = 1
    else:
        doc["file_npm"] = 0
        
    if "requirement_parser" in d:
        doc["file_pypi"] = 1
    else:
        doc["file_pypi"] = 0
        
    if "composer_paser" in d:
        doc["file_composer"] = 1
    else:
        doc["file_composer"] = 0
    
    # npm_tag
    if "npm_tag" in d:
        for t, count in d["npm_tag"].items():
            t = "npm_" + st.stem(t)
            if t in npm_tag_map:
                need_add = True
                npm_tag_map[t] = 1
          
    # pkg_tag
    if "composer_tag" in d:
        for t, count in d["composer_tag"].items():
            t = "pkg_" + st.stem(t)
            if t in pkg_tag_map:
                need_add = True
                pkg_tag_map[t] = 1
                

    # pypi_tag


    if "pypi_tag" in d.keys():

        for t, count in d["pypi_tag"].items():

            t = "pypi_" + st.stem(t)
            if t in pypi_tag_map:
                need_add = True
                pypi_tag_map[t] = 1

    
    if "lang" in d:
        loc_num = 0
        for l in d['lang']:
            if l in lang_map:
                loc_num += d['lang'][l]
                lang_map[l] = d['lang'][l]
        if loc_num != 0:
            lang_map = {k:v/loc_num for k,v in lang_map.items()}
            
    doc.update(lang_map)

    doc.update(npm_tag_map)
    doc.update(pkg_tag_map)
    doc.update(pypi_tag_map)
    doc.update(tag_map)
# remove all null 
    
    if need_add:
        series = pd.Series(doc)
        rec_list.append(series)
    
# rec_pd = pd.DataFrame(rec_list)
("len ", len(rec_list))

100%|██████████| 29368/29368 [00:17<00:00, 1648.93it/s]


('len ', 25467)

In [9]:
len(doc)

451

In [10]:
doc

{'project_link': '/modxcms/evolution',
 'file_readme': 'evolut cm evolut cm requir php gt 5 6what evoevo open sourc content manag system applic framework historyiniti inspir etomit 0 6 modx evolut 0 7 1 0 8 ongo project write raymond irv core team contributor modx evolut cm maintain dmytro lukianenko core team contributor evo project licenseevo distribut gpl licens run profession team develop world visit forum inform featuresevo provid fast lightweight power framework deploy secur websit web applic exampl give true system regist web user group separ administr user grant web user access page access page content manag easili duplic document folder child chunk snippet signific evo s abil empow quickli easili creat maintain rich dynam websit like installjust use evo installerscreenshotsreferencesoffici websit http evo be download http github com evolut cm evolut releasesprevi releas http modx com download evolut previou releas htmlextra http extra evo imdocument http doc evo imhttp evolut 

In [11]:
%%time
rec_pd = pd.DataFrame(rec_list)

CPU times: user 5.74 s, sys: 328 ms, total: 6.06 s
Wall time: 5.96 s


In [12]:
rec_pd.head()

,project_link,file_readme,file_npm,file_pypi,file_composer,JavaScript,HTML,CSS,Python,Shell,...,pytorch,cach,laravel,data-sci,natural-language-process,authent,computer-vis,compos,python3,yii2
0,/freeCodeCamp/freeCodeCamp,welcom freecodecamp org s open sourc codebas c...,1,0,0,0.941868,0.017145,0.039971,0.000000,0.001017,...,0,0,0,0,0,0,0,0,0,0
1,/vuejs/vue,support vue jsvue js mit licens open sourc pro...,1,0,0,0.976735,0.006227,0.003945,0.000000,0.001170,...,0,0,0,0,0,0,0,0,0,0
2,/twbs/bootstrap,bootstrap sleek intuit power end framework fas...,1,0,0,0.494735,0.182466,0.320497,0.000000,0.001512,...,0,0,0,0,0,0,0,0,0,0
3,/facebook/react,react middot react javascript librari build us...,1,0,0,0.951378,0.017036,0.003504,0.000079,0.001854,...,0,0,0,0,0,0,0,0,0,0
4,/airbnb/javascript,airbnb javascript style guid reason approach j...,1,0,0,1.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [13]:
rec_pd.to_csv("../data/data_raw_tagonehot.csv")

In [84]:
doc

{'project_link': '/modxcms/evolution',
 'file_readme': 'evolut cm evolut cm requir php gt 5 6what evoevo open sourc content manag system applic framework historyiniti inspir etomit 0 6 modx evolut 0 7 1 0 8 ongo project write raymond irv core team contributor modx evolut cm maintain dmytro lukianenko core team contributor evo project licenseevo distribut gpl licens run profession team develop world visit forum inform featuresevo provid fast lightweight power framework deploy secur websit web applic exampl give true system regist web user group separ administr user grant web user access page access page content manag easili duplic document folder child chunk snippet signific evo s abil empow quickli easili creat maintain rich dynam websit like installjust use evo installerscreenshotsreferencesoffici websit http evo be download http github com evolut cm evolut releasesprevi releas http modx com download evolut previou releas htmlextra http extra evo imdocument http doc evo imhttp evolut 